# Final Assignment
The following script will load data regarding COVID-19-related deaths grouped by vaccination status and age, and briefly visualize and analyze potential differences in death rates given different vaccination status and age groups.
Obtained from the Government of Ontario Data Catalogue, the link to the CSV file can be found on README.md  

## Importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

## Importing data set and summarizing
First, we import the CSV file using pandas and visualize the first few rows

In [3]:
# Load the dataset
dataset = pd.read_csv('2aa6e2ce-40de-4910-a737-81762e014b0b.csv')

# Display the first few rows of the dataset
dataset.head()

,_id,date,age_group,deaths_boost_vac_rate_7ma,deaths_full_vac_rate_7ma,deaths_not_full_vac_rate_7ma
0,8858,2021-03-01T00:00:00,0-4yrs,0.0,0.0,0.00
1,8859,2021-03-01T00:00:00,5-11yrs,0.0,0.0,0.00
2,8860,2021-03-01T00:00:00,12-17yrs,0.0,0.0,0.00
3,8861,2021-03-01T00:00:00,18-39yrs,0.0,0.0,0.00
4,8862,2021-03-01T00:00:00,40-59yrs,0.0,0.0,0.02
